#### **Import and Convert to Parquet**

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('sales_data.xlsx')
df.shape

(128975, 23)

In [3]:
df.head(2)

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,0.0,Easy Ship
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,Blouse,...,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,0.0,Easy Ship


In [4]:
df.columns = [col.replace('-', ' ').title() for col in df.columns]
df['Date'] = pd.to_datetime(df['Date'])
df['Ship City'] = df['Ship City'].astype('str')
df.to_parquet("sales.parquet")

C:\Users\antsharma\AppData\Local\Temp\ipykernel_20860\764547939.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [5]:
df = pd.read_parquet("sales.parquet")
df.head(2)

,Index,Order Id,Date,Status,Fulfilment,Sales Channel,Ship Service Level,Style,Sku,Category,...,Qty,Currency,Amount,Ship City,Ship State,Ship Postal Code,Ship Country,Promotion Ids,B2B,Fulfilled By
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,None,0.0,Easy Ship
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,Blouse,...,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,0.0,Easy Ship


In [6]:
df.dropna(subset = ['Amount'], inplace = True)
df.shape

(121180, 23)

#### **Q1: How many sales have they made with amounts more than 1000**

In [7]:
len(df[df['Amount'] > 1000])

13332

#### **Q2: How many sales have they made that belong to the Category "Tops" and have a Quantity of 3.**

In [8]:
len(df[(df['Category'] == 'Top') & (df['Qty'] == 3)])

3

#### **Q3: The Total Sales by Category**

In [9]:
df.groupby('Category', as_index = False)\
    .agg({'Amount':'sum'})\
    .sort_values('Amount', ascending = False)\
    .reset_index(drop=True)

,Category,Amount
0,Set,39204124.03
1,Blouse,21757954.88
2,Dress,12131224.11
3,Top,5347792.30
4,Bottom,150667.98
5,Scarf,915.00


#### **Q4: Average Amount by Category and Status**

In [10]:
df.groupby(['Category', 'Fulfilment'], as_index = False)\
    .agg({'Amount': 'mean'})\
    .sort_values('Amount', ascending = False)\
    .reset_index(drop = True)

,Category,Fulfilment,Amount
0,Set,Amazon,838.042180
1,Set,Merchant,822.466512
2,Dress,Amazon,776.077009
3,Dress,Merchant,740.499730
4,Top,Amazon,536.449448
5,Top,Merchant,493.678577
6,Blouse,Amazon,459.493705
7,Blouse,Merchant,451.327641
8,Bottom,Amazon,365.080508
9,Bottom,Merchant,350.592283


#### **Q5: Average Amount by Category and Status**

In [11]:
df.groupby(['Category', 'Status'], as_index = False)\
    .agg({'Amount': 'mean'})\
    .sort_values('Amount', ascending = False)\
    .reset_index(drop = True)\
    .head()

,Category,Status,Amount
0,Dress,Shipped - Damaged,1136.000000
1,Set,Shipped - Out for Delivery,908.105263
2,Dress,Shipped - Rejected by Buyer,899.000000
3,Set,Shipped - Returning to Seller,875.835616
4,Set,Shipped - Picked Up,846.322660


#### **Q6: Total Sales by Fulfilment and Shipment Type**

In [12]:
df.groupby(['Courier Status', 'Fulfilment'], as_index = False)\
    .agg({'Amount': 'sum'})\
    .sort_values('Amount', ascending = False)\
    .reset_index(drop = True)

,Courier Status,Fulfilment,Amount
0,Shipped,Amazon,50331934.0
1,Shipped,Merchant,20726730.0
2,Unshipped,Amazon,3990217.0
3,Unshipped,Merchant,353625.0
